In [ ]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau

In [ ]:
# Set random seed for reproducibility
random.seed(42)

In [ ]:
def walk_through_dir(dir_path):
  """Walks through the directory and returns it's content"""
  for dir_path, dirname, filenames in os.walk(dir_path):
    print(f"There are {len(dirname)} directories and {len(filenames)} images in {dir_path}")

walk_through_dir("D:\\Ashutosh\\Herbs\\Cleanede_Data")

In [ ]:
IMAGE_PATH = "D:\\Ashutosh\\Herbs\\Cleanede_Data"

# Get all image paths
image_path_list = list(Path(IMAGE_PATH).glob("**/*/*"))
rand_image = random.choice(image_path_list)
image_class = rand_image.parent.stem

# Open and display a random image
img = Image.open(rand_image)
print(f"The image path is {rand_image} and the width and height are {img.width} {img.height}")
img.show()  # Uncomment if you want to display the image

In [ ]:
def apply_random_augmentation(image):
    """Apply random augmentations to an image"""
    image = tf.convert_to_tensor(np.array(image), dtype=tf.float32)
    if tf.reduce_max(image) > 1.0:
        image = image / 255.0
    
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_hue(image, max_delta=0.1)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    
    image = tf.clip_by_value(image, 0.0, 1.0)
    image = (image * 255).astype(np.uint8)
    return image

In [ ]:
def plot_transformed_image(img_paths, transform, n=3, seed=42):
    """Plot original and transformed images"""
    random.seed(seed)
    random_image_paths = random.sample(img_paths, k=n)
    
    for image_path in random_image_paths:
        with Image.open(image_path) as f:
            fig, ax = plt.subplots(figsize=(12, 5), nrows=1, ncols=2)
            ax[0].imshow(f)
            ax[0].set_title(f"Original image with {f.size}")
            ax[0].axis('off')
            
            transformed_image = transform(f)
            ax[1].imshow(transformed_image)
            ax[1].set_title("Transformed image")
            ax[1].axis('off')
            
            fig.suptitle(f"Class: {image_path.parent.stem}")
            plt.show()

# Plot some example augmentations
plot_transformed_image(img_paths=image_path_list, transform=apply_random_augmentation, n=3, seed=42)

In [ ]:
def create_folder(fd):
    if not os.path.exists(fd):
        os.makedirs(fd)

output = 'D:\\Ashutosh\\Herbs\\Output\\ModelName'  
create_folder(output)

In [ ]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

In [ ]:
# Only rescaling for validation
validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Set image size and batch size
image_size = (224, 224)
batch_size = 32

In [ ]:
train_directory = 'D:\\Ashutosh\\Herbs\\Cleanede_Data\\Train'
validation_directory = 'D:\\Ashutosh\\Herbs\\Cleanede_Data\\Val'

In [ ]:
# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
# Create DenseNet121 model
base_model = DenseNet121(
    include_top=False,
    input_shape=(image_size[0], image_size[1], 3),
    weights='imagenet'
)

model_dense = keras.models.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(109, activation='softmax')  # Verify this matches your number of classes
])

model_dense.summary()

In [ ]:
# Define callbacks
filepath = os.path.join(output, "HerbClassification_BEST.h5")
csv_logger = CSVLogger(os.path.join(output, f"CSV_Logger-MC_ML-{time.time()}.csv"))
early_stop = EarlyStopping(patience=15, monitor='val_loss', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.0001, verbose=1)
checkpoint = ModelCheckpoint(
    filepath,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='max'
)

callbacks_list = [checkpoint, reduce_lr, early_stop, csv_logger]

In [ ]:
# Compile model
model_dense.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossentropy(),
    metrics=['accuracy']
)

# Calculate steps per epoch
steps_per_epoch = train_generator.samples // batch_size
validation_steps = validation_generator.samples // batch_size

In [ ]:
# Train model
epochs = 200
densenet_history = model_dense.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=callbacks_list,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

In [ ]:
# Save final weights
model_dense.save_weights(os.path.join(output, 'herbs_100Epoch_Last_WT.h5'))